In [2]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from pydub import AudioSegment

# Load full audio files
file_path1 = "../audio/test_clip_1.mp3"
file_path2 = "../audio/test_clip_2.mp3"
y1, sr1 = librosa.load(file_path1)
y2, sr2 = librosa.load(file_path2)

In [3]:
transition_duration = 5  # seconds
frames_A = int(transition_duration * sr1)
frames_B = int(transition_duration * sr2)
segment_A = y1[-frames_A:]
segment_B = y2[:frames_B]

In [4]:
def compute_complexity(y_segment, sr):
    onset_env = librosa.onset.onset_strength(y=y_segment, sr=sr)
    return np.std(onset_env)

complexity_A = compute_complexity(segment_A, sr1)
complexity_B = compute_complexity(segment_B, sr2)
print(f"Complexity A (end of song 1): {complexity_A:.4f}")
print(f"Complexity B (start of song 2): {complexity_B:.4f}")

Complexity A (end of song 1): 2.5075
Complexity B (start of song 2): 2.8112


In [5]:
hop_length = 512
if complexity_A < complexity_B:
    rate = complexity_B / complexity_A
    D = librosa.stft(segment_A)
    D_stretched = librosa.phase_vocoder(D, rate=rate, hop_length=hop_length)
    stretched_segment = librosa.istft(D_stretched, hop_length=hop_length)
    sf.write("transition_stretched.wav", stretched_segment, sr1)
    sf.write("other_segment.wav", segment_B, sr2)
    simpler_first = True
else:
    rate = complexity_A / complexity_B
    D = librosa.stft(segment_B)
    D_stretched = librosa.phase_vocoder(D, rate=rate, hop_length=hop_length)
    stretched_segment = librosa.istft(D_stretched, hop_length=hop_length)
    sf.write("transition_stretched.wav", stretched_segment, sr2)
    sf.write("other_segment.wav", segment_A, sr1)
    simpler_first = False

In [6]:
stretched_audio = AudioSegment.from_wav("transition_stretched.wav")
other_audio = AudioSegment.from_wav("other_segment.wav")
crossfade = stretched_audio.append(other_audio, crossfade=3000)
crossfade.export("../audio/transition_output.mp3", format="mp3")

<_io.BufferedRandom name='../audio/transition_output.mp3'>

In [7]:
full_song1 = AudioSegment.from_file("../audio/song1.wav")
full_song2 = AudioSegment.from_file("../audio/song2_stretched.wav")

if simpler_first:
    song2_cut = full_song2[5000:]  # remove 5s from start if already in transition
    final_mix = full_song1 + crossfade + song2_cut
else:
    song1_cut = full_song1[:-5000]  # remove 5s from end if already in transition
    final_mix = song1_cut + crossfade + full_song2

final_mix.export("../audio/final_complexity_mix.mp3", format="mp3")
print("✅ Final mix saved to audio/final_complexity_mix.mp3")

✅ Final mix saved to audio/final_complexity_mix.mp3
